In [1]:
import os
import time
import numpy as np
import pandas as pd
from dataclasses import dataclass
from datetime import datetime, timedelta

# ============================================================
# CONFIG
# ============================================================

BASE_DIR = r"D:\work\Trade Analysis\Mapping ExERCISE"

# Iteration list file inside BASE_DIR
ITERATION_FILE_PATH = os.path.join(BASE_DIR, "Iteration_list_2026_1_20_20.xlsx")  # change if needed

# Price data folder inside BASE_DIR (ACN.csv, ABN.csv, ...)
PRICE_DIR = "D:\work\Trade Analysis\Mapping ExERCISE\Latest_Data_L"  # or os.path.join(BASE_DIR, "Price")

# Outputs / State (inside BASE_DIR)
OUT_DIR            = os.path.join(BASE_DIR, "output")
STATE_HISTORY_PATH = os.path.join(BASE_DIR, "state_history.csv")
os.makedirs(OUT_DIR, exist_ok=True)

# Market settings
MARKET_START  = "14:30"
MARKET_END    = "21:00"
RESAMPLE_RULE = "15T"

# Model params
ATR_PERIOD = 14
REG_WINDOW = 16

TRIGGER_ON_CLOSE  = True
ATR_FALLBACK_MULT = 3.0

# Initial SL must be >= 1%
INITIAL_MIN_STOP_PCT  = 0.01   # 1%
TRAILING_MIN_STOP_PCT = 0.001  # 0.10%

APPLY_FROM_NEXT_DAY_ONLY = True

# Runner
POLL_SECONDS = 15 * 60
MODE = "once"  # "once" or "loop"

# Best params
CFG = {
    "ATAN_LONG": 0.0015,
    "ATAN_SHORT": 0.0013,
    "MAX_STOP_LONG": 0.012,
    "MAX_STOP_SHORT": 0.012,
    "SENS_LONG": 1.0,
    "SENS_SHORT": 1.0,
}

# Default direction (if not present)
DEFAULT_DIRECTION = "LONG"

STATUS_OPEN_VALUES   = {"OPEN", "NEW"}
STATUS_ACTIVE_VALUES = {"ACTIVE", "RUNNING"}
STATUS_CLOSED_VALUES = {"CLOSED", "EXITED", "DONE"}

# ============================================================
# Helpers
# ============================================================

def now_ts():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def _todelta_hhmm(hhmm: str) -> pd.Timedelta:
    return pd.to_timedelta(hhmm + ":00")

def _norm_col(s: str) -> str:
    return "".join(ch for ch in str(s).strip().lower() if ch.isalnum())

def find_column(df: pd.DataFrame, candidates: list[str]) -> str | None:
    col_map = {_norm_col(c): c for c in df.columns}
    for cand in candidates:
        key = _norm_col(cand)
        if key in col_map:
            return col_map[key]
    return None

def normalize_symbol(x: str) -> str:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ""
    return str(x).strip().upper()

def status_bucket(s: str) -> str:
    s = (s or "").strip().upper()
    if s in STATUS_CLOSED_VALUES:
        return "CLOSED"
    if s in STATUS_ACTIVE_VALUES:
        return "ACTIVE"
    if s in STATUS_OPEN_VALUES:
        return "OPEN"
    return "ACTIVE"

def normalize_direction(x) -> str:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return "UNKNOWN"
    s = str(x).strip().upper()
    if s in ["LONG", "BUY", "B", "1", "1.0", "+1", "+1.0"]:
        return "LONG"
    if s in ["SHORT", "SELL", "S", "-1", "-1.0"]:
        return "SHORT"
    try:
        v = float(s)
        if v > 0: return "LONG"
        if v < 0: return "SHORT"
    except Exception:
        pass
    return "UNKNOWN"

def yesterday_date() -> pd.Timestamp:
    # "yesterday" in local system time
    return (pd.Timestamp.now().normalize() - pd.Timedelta(days=1))

# ============================================================
# Iteration loader (uses latest price, no EntryDate required)
# ============================================================

ITER_KEY_CANDS = ["created by final", "created_by_final", "Created By Final", "Created_By_Final"]
ITER_COLS = {
    "Currency":   ["Currency", "currency"],
    "Status":     ["Status", "status", "TradeStatus", "State"],
    "ORI_symbol": ["ORI_symbol", "ORI Symbol", "ori_symbol", "OriginalSymbol", "Original Symbol"],
    "FXCM_currency_name": ["FXCM_Currency_Name", "FXCM_currency_name", "FXCM Currency Name"],
    "Condition":  ["Condition", "condition"],
    "Direction":  ["Direction", "direction", "Side", "side", "BuySell", "buy_sell", "Signal", "signal"],
    "LatestPrice": ["latest price", "Latest Price", "latest_price", "LatestPrice", "last price", "Last Price", "last_price"],
}

def load_iteration_df(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Iteration file not found: {path}")

    df = pd.read_excel(path) if path.lower().endswith((".xlsx", ".xls")) else pd.read_csv(path)
    df.columns = [c.strip() for c in df.columns]

    key_col = find_column(df, ITER_KEY_CANDS)
    if key_col is None:
        raise ValueError(f"Cannot find unique key column (created by final). Columns: {list(df.columns)}")

    found = {k: find_column(df, cands) for k, cands in ITER_COLS.items()}

    # Must have: Status, ori_symbol, latest price
    req = ["Status", "ORI_symbol", "LatestPrice"]
    missing = [k for k in req if not found.get(k)]
    if missing:
        raise ValueError(
            f"Iteration list missing required columns {missing}. "
            f"Found mapping: {found}. Columns: {list(df.columns)}"
        )

    out = pd.DataFrame()
    out["TradeKey"]   = df[key_col].astype(str).str.strip()
    out["ORI_symbol"] = df[found["ORI_symbol"]].apply(normalize_symbol)
    out["Status"]     = df[found["Status"]].astype(str).str.strip().str.upper().apply(status_bucket)
    out["EntryPrice"] = pd.to_numeric(df[found["LatestPrice"]], errors="coerce")

    if found.get("Direction"):
        out["Direction"] = df[found["Direction"]].apply(normalize_direction)
        out.loc[~out["Direction"].isin(["LONG","SHORT"]), "Direction"] = DEFAULT_DIRECTION
    else:
        out["Direction"] = DEFAULT_DIRECTION

    out["Currency"] = df[found["Currency"]].astype(str) if found.get("Currency") else ""
    out["FXCM_Currency_Name"] = df[found["FXCM_currency_name"]].astype(str) if found.get("FXCM_currency_name") else ""
    out["Condition"] = df[found["Condition"]].astype(str) if found.get("Condition") else ""

    # EntryDate rule (your assumption)
    # OPEN => yesterday; ACTIVE => keep from state if exists else yesterday (done later)
    out["EntryDate_Assumed"] = yesterday_date()

    # clean
    out = out[out["TradeKey"].notna() & (out["TradeKey"].str.len() > 0)].copy()
    out = out[out["ORI_symbol"].notna() & (out["ORI_symbol"].str.len() > 0)].copy()
    out = out[out["EntryPrice"].notna()].copy()
    out = out.sort_values(["TradeKey"]).drop_duplicates(subset=["TradeKey"], keep="last").reset_index(drop=True)

    print(f"[{now_ts()}] Iteration trades loaded: {len(out)}")
    return out

# ============================================================
# Price file loading from folder
# ============================================================

PRICE_COL_CANDS = {
    "date":  ["date", "Date", "datetime", "Datetime", "timestamp", "Timestamp"],
    "open":  ["open", "Open"],
    "high":  ["high", "High"],
    "low":   ["low", "Low"],
    "close": ["close", "Close"],
}

def resolve_price_file(symbol: str) -> str | None:
    sym = normalize_symbol(symbol)
    for ext in [".csv", ".CSV", ".parquet", ".PARQUET"]:
        fp = os.path.join(PRICE_DIR, f"{sym}{ext}")
        if os.path.exists(fp):
            return fp
    # fallback: startswith sym
    for fn in os.listdir(PRICE_DIR):
        if fn.upper().startswith(sym + "."):
            fp = os.path.join(PRICE_DIR, fn)
            if os.path.isfile(fp):
                return fp
    return None

def load_price_file(fp: str) -> pd.DataFrame:
    df = pd.read_parquet(fp) if fp.lower().endswith(".parquet") else pd.read_csv(fp)
    df.columns = [c.strip() for c in df.columns]
    cols = {k: find_column(df, v) for k, v in PRICE_COL_CANDS.items()}
    missing = [k for k, v in cols.items() if v is None]
    if missing:
        raise ValueError(f"Price file {os.path.basename(fp)} missing {missing}. Columns: {list(df.columns)}")

    out = pd.DataFrame()
    out["_dt"] = pd.to_datetime(df[cols["date"]], errors="coerce")
    for c in ["open","high","low","close"]:
        out[c] = pd.to_numeric(df[cols[c]], errors="coerce")
    out = out.dropna(subset=["_dt","open","high","low","close"]).sort_values("_dt").reset_index(drop=True)
    return out

def filter_market_hours(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    dfi = df.set_index("_dt", drop=False).sort_index()
    dfi = dfi.between_time(MARKET_START, MARKET_END, inclusive="both")
    return dfi.reset_index(drop=True)

def resample_ohlc(df: pd.DataFrame, rule: str) -> pd.DataFrame:
    if df.empty:
        return df
    dfi = df.set_index("_dt").sort_index()
    ohlc = dfi[["open","high","low","close"]].resample(rule).agg({
        "open": "first",
        "high": "max",
        "low": "min",
        "close": "last",
    }).dropna()
    return ohlc.reset_index()

def compute_atr(df: pd.DataFrame, period: int) -> pd.Series:
    high = df["high"]
    low = df["low"]
    close = df["close"]
    tr = pd.concat([
        high - low,
        (high - close.shift(1)).abs(),
        (low - close.shift(1)).abs()
    ], axis=1).max(axis=1)
    return tr.rolling(period, min_periods=period).mean()

_RS_CACHE = {}  # symbol -> rs (15T)

def prepare_rs_for_symbol(symbol: str) -> pd.DataFrame | None:
    sym = normalize_symbol(symbol)
    if sym in _RS_CACHE:
        return _RS_CACHE[sym]

    fp = resolve_price_file(sym)
    if fp is None:
        _RS_CACHE[sym] = None
        return None

    df = load_price_file(fp)
    df = filter_market_hours(df)
    if df.empty:
        _RS_CACHE[sym] = None
        return None

    rs = resample_ohlc(df, RESAMPLE_RULE)
    if rs.empty:
        _RS_CACHE[sym] = None
        return None

    rs["ATR"] = compute_atr(rs, ATR_PERIOD)
    _RS_CACHE[sym] = rs
    return rs

# ============================================================
# Stop logic
# ============================================================

def adaptive_stop_distance_pct(bars: pd.DataFrame,
                               atan_threshold: float,
                               atr_fallback_mult: float,
                               min_stop_pct: float,
                               max_stop_pct: float,
                               sensitivity: float) -> float:
    if len(bars) < REG_WINDOW or bars["ATR"].dropna().empty:
        return np.nan

    recent = bars["close"].iloc[-REG_WINDOW:].astype(float)
    if len(recent) < REG_WINDOW:
        return np.nan

    x = np.arange(len(recent), dtype=float)
    a, b, _c = np.polyfit(x, recent.values, 2)

    slope_norm = (2 * a * x[-1] + b) / recent.iloc[-1]
    accel = (2 * a)
    atan_slope = float(np.arctan(slope_norm))

    atr = bars["ATR"].iloc[-1]
    px = bars["close"].iloc[-1]
    atr_pct = float(atr / px) if (pd.notna(atr) and atr > 0 and px > 0) else np.nan
    if pd.isna(atr_pct):
        return np.nan

    if abs(atan_slope) > atan_threshold:
        stop_pct = abs(slope_norm) * sensitivity * (1.0 + abs(accel))
        return float(np.clip(stop_pct, min_stop_pct, max_stop_pct))

    stop_pct = atr_pct * atr_fallback_mult
    return float(np.clip(stop_pct, min_stop_pct, max_stop_pct))

def compute_initial_stop_pct(rs: pd.DataFrame, direction: str) -> float:
    is_long = (normalize_direction(direction) == "LONG")
    atan_thr = CFG["ATAN_LONG"] if is_long else CFG["ATAN_SHORT"]
    max_stop = CFG["MAX_STOP_LONG"] if is_long else CFG["MAX_STOP_SHORT"]
    sens     = CFG["SENS_LONG"] if is_long else CFG["SENS_SHORT"]

    stop_pct = adaptive_stop_distance_pct(
        rs, atan_thr, ATR_FALLBACK_MULT,
        INITIAL_MIN_STOP_PCT, max_stop, sens
    )
    if pd.notna(stop_pct):
        return float(stop_pct)

    return float(np.clip(INITIAL_MIN_STOP_PCT, INITIAL_MIN_STOP_PCT, max_stop))

def init_stop_from_entry(entry_px: float, direction: str, stop_pct: float) -> float:
    is_long = (normalize_direction(direction) == "LONG")
    return entry_px * (1.0 - stop_pct) if is_long else entry_px * (1.0 + stop_pct)

def compute_monitor_start(entry_dt: pd.Timestamp) -> pd.Timestamp:
    if not APPLY_FROM_NEXT_DAY_ONLY:
        return entry_dt
    return entry_dt.normalize() + timedelta(days=1) + _todelta_hhmm(MARKET_START)

# ============================================================
# State history
# ============================================================

STATE_COLS = [
    "TradeKey","ORI_symbol","Direction",
    "EntryDate","EntryPrice",
    "Status","StopPrice","StopPct","StopSource",
    "BestRefPrice","LastBarDT"
]

def load_state_history(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame(columns=STATE_COLS)
    df = pd.read_csv(path)
    df.columns = [c.strip() for c in df.columns]
    if "EntryDate" in df.columns:
        df["EntryDate"] = pd.to_datetime(df["EntryDate"], errors="coerce")
    if "LastBarDT" in df.columns:
        df["LastBarDT"] = pd.to_datetime(df["LastBarDT"], errors="coerce")
    return df

def save_state_history(df: pd.DataFrame, path: str):
    df.to_csv(path, index=False)

# ============================================================
# Actions
# ============================================================

@dataclass
class Action:
    TradeKey: str
    ORI_symbol: str
    ActionType: str      # INIT_STOP / UPDATE_STOP / EXIT / NO_ACTION
    ActionTime: str
    StopPrice: float | None
    StopPct: float | None
    Reason: str

# ============================================================
# Per-trade step
# ============================================================

def process_trade_one_step(tr_state: pd.Series, rs: pd.DataFrame) -> tuple[pd.Series, list[Action]]:
    actions = []
    tkey = tr_state["TradeKey"]
    sym  = tr_state["ORI_symbol"]
    direction = normalize_direction(tr_state["Direction"])
    is_long = (direction == "LONG")

    entry_dt = pd.to_datetime(tr_state["EntryDate"])
    entry_px = float(tr_state["EntryPrice"])

    stop_price = float(tr_state["StopPrice"])
    stop_pct   = float(tr_state["StopPct"]) if pd.notna(tr_state["StopPct"]) else np.nan
    stop_src   = str(tr_state["StopSource"]) if pd.notna(tr_state["StopSource"]) else "Initial"
    best_ref   = float(tr_state["BestRefPrice"]) if pd.notna(tr_state["BestRefPrice"]) else entry_px
    last_dt    = pd.to_datetime(tr_state["LastBarDT"]) if pd.notna(tr_state["LastBarDT"]) else None

    rs = rs.sort_values("_dt").reset_index(drop=True)

    start_dt = compute_monitor_start(entry_dt)
    new_bars = rs[rs["_dt"] >= start_dt].copy() if last_dt is None else rs[rs["_dt"] > last_dt].copy()

    if new_bars.empty:
        actions.append(Action(tkey, sym, "NO_ACTION", now_ts(), stop_price, stop_pct, "No new bars"))
        return tr_state, actions

    exited = False
    updated = False
    last_processed = None

    for dt_val in new_bars["_dt"].tolist():
        idx = rs.index[rs["_dt"] == dt_val]
        if len(idx) == 0:
            continue
        i = int(idx[0])
        sub = rs.iloc[:i+1]
        row = rs.iloc[i]
        dt = row["_dt"]
        last_processed = dt

        close = float(row["close"])
        high  = float(row["high"])
        low   = float(row["low"])

        if is_long:
            best_ref = max(best_ref, close)
            hit_val = close if TRIGGER_ON_CLOSE else low
            if hit_val <= stop_price:
                tr_state["Status"] = "CLOSED"
                tr_state["LastBarDT"] = dt
                actions.append(Action(tkey, sym, "EXIT", str(dt), stop_price, stop_pct, f"Stop hit ({stop_src})"))
                exited = True
                break
        else:
            best_ref = min(best_ref, close)
            hit_val = close if TRIGGER_ON_CLOSE else high
            if hit_val >= stop_price:
                tr_state["Status"] = "CLOSED"
                tr_state["LastBarDT"] = dt
                actions.append(Action(tkey, sym, "EXIT", str(dt), stop_price, stop_pct, f"Stop hit ({stop_src})"))
                exited = True
                break

        atan_thr = CFG["ATAN_LONG"] if is_long else CFG["ATAN_SHORT"]
        max_stop = CFG["MAX_STOP_LONG"] if is_long else CFG["MAX_STOP_SHORT"]
        sens     = CFG["SENS_LONG"] if is_long else CFG["SENS_SHORT"]

        new_stop_pct = adaptive_stop_distance_pct(
            sub, atan_thr, ATR_FALLBACK_MULT,
            TRAILING_MIN_STOP_PCT, max_stop, sens
        )
        if pd.isna(new_stop_pct):
            continue

        candidate_stop = (best_ref * (1.0 - new_stop_pct)) if is_long else (best_ref * (1.0 + new_stop_pct))

        if is_long and candidate_stop > stop_price:
            stop_price = float(candidate_stop)
            stop_pct = float(new_stop_pct)
            stop_src = "Trailing"
            updated = True
        elif (not is_long) and candidate_stop < stop_price:
            stop_price = float(candidate_stop)
            stop_pct = float(new_stop_pct)
            stop_src = "Trailing"
            updated = True

    tr_state["StopPrice"] = stop_price
    tr_state["StopPct"] = stop_pct
    tr_state["StopSource"] = stop_src
    tr_state["BestRefPrice"] = best_ref
    if last_processed is not None:
        tr_state["LastBarDT"] = last_processed

    if exited:
        return tr_state, actions

    if updated:
        actions.append(Action(tkey, sym, "UPDATE_STOP", now_ts(), stop_price, stop_pct, "Stop updated"))
    else:
        actions.append(Action(tkey, sym, "NO_ACTION", now_ts(), stop_price, stop_pct, "Stop unchanged"))

    return tr_state, actions

# ============================================================
# Runner
# ============================================================

def main_once():
    print(f"[{now_ts()}] Live Trailing Runner start")
    print(f"[{now_ts()}] BASE_DIR={BASE_DIR}")
    print(f"[{now_ts()}] PRICE_DIR={PRICE_DIR}")
    print(f"[{now_ts()}] OUT_DIR={OUT_DIR}")
    print(f"[{now_ts()}] STATE_HISTORY={STATE_HISTORY_PATH}")

    iter_df = load_iteration_df(ITERATION_FILE_PATH)
    st_hist = load_state_history(STATE_HISTORY_PATH)

    # preload rs per symbol
    missing_files = []
    for sym in sorted(iter_df["ORI_symbol"].unique().tolist()):
        if resolve_price_file(sym) is None:
            missing_files.append(sym)
        prepare_rs_for_symbol(sym)

    if missing_files:
        print(f"[{now_ts()}] WARNING: missing price files for symbols (first 25): {missing_files[:25]}")

    state_rows = []
    actions_all = []

    for _, tr in iter_df.iterrows():
        tkey = tr["TradeKey"]
        sym  = tr["ORI_symbol"]
        direction = tr["Direction"]
        status = tr["Status"]

        rs = prepare_rs_for_symbol(sym)
        if rs is None or rs.empty:
            actions_all.append(Action(tkey, sym, "NO_ACTION", now_ts(), None, None, "Missing price data for symbol"))
            continue

        prev = st_hist[st_hist["TradeKey"] == tkey].copy()

        if not prev.empty:
            row = prev.iloc[-1].copy()

            # refresh always
            row["ORI_symbol"] = sym
            row["Status"] = status
            row["Direction"] = direction if direction in ["LONG","SHORT"] else row.get("Direction", DEFAULT_DIRECTION)

            # Update EntryPrice daily from latest price (your rule)
            row["EntryPrice"] = float(tr["EntryPrice"])

            # EntryDate handling: keep existing; if missing, set yesterday
            if pd.isna(row.get("EntryDate", pd.NaT)):
                row["EntryDate"] = yesterday_date()

        else:
            # first time seen: create state
            # EntryDate assumption: OPEN => yesterday; ACTIVE => yesterday (bootstrap)
            entry_dt = yesterday_date()

            row = pd.Series({
                "TradeKey": tkey,
                "ORI_symbol": sym,
                "Direction": direction if direction in ["LONG","SHORT"] else DEFAULT_DIRECTION,
                "EntryDate": entry_dt,
                "EntryPrice": float(tr["EntryPrice"]),
                "Status": status,
                "StopPrice": np.nan,
                "StopPct": np.nan,
                "StopSource": "",
                "BestRefPrice": float(tr["EntryPrice"]),
                "LastBarDT": pd.NaT
            })

        # init initial stop if missing
        if pd.isna(row.get("StopPrice", np.nan)) or (row.get("StopPrice", None) is None):
            stop_pct = compute_initial_stop_pct(rs, row["Direction"])
            stop_price = init_stop_from_entry(float(row["EntryPrice"]), row["Direction"], stop_pct)

            row["StopPrice"] = float(stop_price)
            row["StopPct"] = float(stop_pct)
            row["StopSource"] = "Initial"
            row["BestRefPrice"] = float(row["EntryPrice"])
            row["LastBarDT"] = pd.NaT

            actions_all.append(Action(
                tkey, sym, "INIT_STOP", now_ts(),
                float(stop_price), float(stop_pct),
                "Initial stop (>=1%) from latest price"
            ))

        # process if not closed
        if status_bucket(row["Status"]) != "CLOSED":
            row2, acts = process_trade_one_step(row.copy(), rs)
            actions_all.extend(acts)
            state_rows.append(row2)
        else:
            state_rows.append(row)

    updated_state_df = pd.DataFrame(state_rows)

    # merge: replace OPEN/ACTIVE rows present today, keep historical CLOSED
    if not st_hist.empty and not updated_state_df.empty:
        open_active_mask = st_hist["Status"].astype(str).str.upper().isin(["OPEN","ACTIVE"])
        st_hist = st_hist[~(open_active_mask & st_hist["TradeKey"].isin(updated_state_df["TradeKey"]))].copy()

    final_state = pd.concat([st_hist, updated_state_df], ignore_index=True)

    actions_df = pd.DataFrame([a.__dict__ for a in actions_all])
    actions_path = os.path.join(OUT_DIR, f"actions_{datetime.now().strftime('%Y%m%d')}.csv")
    state_path   = os.path.join(OUT_DIR, f"state_snapshot_{datetime.now().strftime('%Y%m%d')}.csv")

    if os.path.exists(actions_path):
        actions_df.to_csv(actions_path, mode="a", header=False, index=False)
    else:
        actions_df.to_csv(actions_path, index=False)

    final_state.to_csv(state_path, index=False)
    save_state_history(final_state, STATE_HISTORY_PATH)

    print(f"[{now_ts()}] Saved actions: {actions_path}")
    print(f"[{now_ts()}] Saved state snapshot: {state_path}")
    print(f"[{now_ts()}] Updated persistent state: {STATE_HISTORY_PATH}")
    print(f"[{now_ts()}] Runner done")

def main():
    if MODE == "once":
        main_once()
    else:
        while True:
            try:
                main_once()
            except Exception as e:
                print(f"[{now_ts()}] ERROR: {e}")
            time.sleep(POLL_SECONDS)

if __name__ == "__main__":
    main()


[2026-01-24 00:10:18] Live Trailing Runner start
[2026-01-24 00:10:18] BASE_DIR=D:\work\Trade Analysis\Mapping ExERCISE
[2026-01-24 00:10:18] PRICE_DIR=D:\work\Trade Analysis\Mapping ExERCISE\Latest_Data_L
[2026-01-24 00:10:18] OUT_DIR=D:\work\Trade Analysis\Mapping ExERCISE\output
[2026-01-24 00:10:18] STATE_HISTORY=D:\work\Trade Analysis\Mapping ExERCISE\state_history.csv
[2026-01-24 00:10:19] Iteration trades loaded: 329


C:\Users\avina\AppData\Local\Temp\ipykernel_8352\3656859521.py:238: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ohlc = dfi[["open","high","low","close"]].resample(rule).agg({
C:\Users\avina\AppData\Local\Temp\ipykernel_8352\3656859521.py:238: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ohlc = dfi[["open","high","low","close"]].resample(rule).agg({
C:\Users\avina\AppData\Local\Temp\ipykernel_8352\3656859521.py:238: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ohlc = dfi[["open","high","low","close"]].resample(rule).agg({
C:\Users\avina\AppData\Local\Temp\ipykernel_8352\3656859521.py:238: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ohlc = dfi[["open","high","low","close"]].resample(rule).agg({
C:\Users\avina\AppData\Local\Temp\ipykernel_8352\3656859521.py:238: 

[2026-01-24 00:10:20] Saved actions: D:\work\Trade Analysis\Mapping ExERCISE\output\actions_20260124.csv
[2026-01-24 00:10:20] Saved state snapshot: D:\work\Trade Analysis\Mapping ExERCISE\output\state_snapshot_20260124.csv
[2026-01-24 00:10:20] Updated persistent state: D:\work\Trade Analysis\Mapping ExERCISE\state_history.csv
[2026-01-24 00:10:20] Runner done


In [ ]:
import os
import time
import numpy as np
import pandas as pd
from dataclasses import dataclass
from datetime import datetime, timedelta

# ============================================================
# CONFIG
# ============================================================

BASE_DIR = r"D:\work\Trade Analysis\Mapping ExERCISE"

ITERATION_FILE_PATH = os.path.join(BASE_DIR, "Iteration_list_2026_1_20_20.xlsx")  # change if needed
PRICE_DIR = "D:\work\Trade Analysis\Mapping ExERCISE\Latest_Data_L"   # ACN.csv, etc.

OUT_DIR            = os.path.join(BASE_DIR, "output")
STATE_HISTORY_PATH = os.path.join(BASE_DIR, "state_history.csv")
LIVE_XLSX_PATH     = os.path.join(OUT_DIR, "live_actions.xlsx")
LOG_PATH           = os.path.join(OUT_DIR, "runner.log")

os.makedirs(OUT_DIR, exist_ok=True)

# Market settings
MARKET_START  = "14:30"
MARKET_END    = "21:00"
RESAMPLE_RULE = "15T"

# Model params
ATR_PERIOD = 14
REG_WINDOW = 16

TRIGGER_ON_CLOSE  = True
ATR_FALLBACK_MULT = 3.0

INITIAL_MIN_STOP_PCT  = 0.01   # >=1%
TRAILING_MIN_STOP_PCT = 0.001  # 0.10%

APPLY_FROM_NEXT_DAY_ONLY = True

# Runner loop
POLL_SECONDS = 15 * 60   # 15 minutes
MODE = "loop"            # "once" or "loop"

# Best params
CFG = {
    "ATAN_LONG": 0.0015,
    "ATAN_SHORT": 0.0013,
    "MAX_STOP_LONG": 0.012,
    "MAX_STOP_SHORT": 0.012,
    "SENS_LONG": 1.0,
    "SENS_SHORT": 1.0,
}

DEFAULT_DIRECTION = "LONG"

STATUS_OPEN_VALUES   = {"OPEN", "NEW"}
STATUS_ACTIVE_VALUES = {"ACTIVE", "RUNNING"}
STATUS_CLOSED_VALUES = {"CLOSED", "EXITED", "DONE"}

# ============================================================
# Logging
# ============================================================

def now_ts():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

def log(msg: str):
    line = f"[{now_ts()}] {msg}"
    print(line)
    with open(LOG_PATH, "a", encoding="utf-8") as f:
        f.write(line + "\n")

# ============================================================
# Helpers
# ============================================================

def _todelta_hhmm(hhmm: str) -> pd.Timedelta:
    return pd.to_timedelta(hhmm + ":00")

def _norm_col(s: str) -> str:
    return "".join(ch for ch in str(s).strip().lower() if ch.isalnum())

def find_column(df: pd.DataFrame, candidates: list[str]) -> str | None:
    col_map = {_norm_col(c): c for c in df.columns}
    for cand in candidates:
        key = _norm_col(cand)
        if key in col_map:
            return col_map[key]
    return None

def normalize_symbol(x: str) -> str:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ""
    return str(x).strip().upper()

def status_bucket(s: str) -> str:
    s = (s or "").strip().upper()
    if s in STATUS_CLOSED_VALUES:
        return "CLOSED"
    if s in STATUS_ACTIVE_VALUES:
        return "ACTIVE"
    if s in STATUS_OPEN_VALUES:
        return "OPEN"
    return "ACTIVE"

def normalize_direction(x) -> str:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return "UNKNOWN"
    s = str(x).strip().upper()
    if s in ["LONG", "BUY", "B", "1", "1.0", "+1", "+1.0"]:
        return "LONG"
    if s in ["SHORT", "SELL", "S", "-1", "-1.0"]:
        return "SHORT"
    try:
        v = float(s)
        if v > 0: return "LONG"
        if v < 0: return "SHORT"
    except Exception:
        pass
    return "UNKNOWN"

def yesterday_date() -> pd.Timestamp:
    return (pd.Timestamp.now().normalize() - pd.Timedelta(days=1))

# ============================================================
# Iteration loader (uses "latest price" ONLY for INIT)
# ============================================================

ITER_KEY_CANDS = ["created by final", "created_by_final", "Created By Final", "Created_By_Final"]
ITER_COLS = {
    "Status":     ["Status", "status", "TradeStatus", "State"],
    "ORI_symbol": ["ORI_symbol", "ORI Symbol", "ori_symbol", "OriginalSymbol", "Original Symbol"],
    "LatestPrice": ["latest price", "Latest Price", "latest_price", "LatestPrice", "last price", "Last Price", "last_price"],
    "Direction":  ["Direction", "direction", "Side", "side", "BuySell", "buy_sell", "Signal", "signal"],
}

def load_iteration_df(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        raise FileNotFoundError(f"Iteration file not found: {path}")

    df = pd.read_excel(path) if path.lower().endswith((".xlsx", ".xls")) else pd.read_csv(path)
    df.columns = [c.strip() for c in df.columns]

    key_col = find_column(df, ITER_KEY_CANDS)
    if key_col is None:
        raise ValueError(f"Cannot find unique key column (created by final). Columns: {list(df.columns)}")

    found = {k: find_column(df, cands) for k, cands in ITER_COLS.items()}
    req = ["Status", "ORI_symbol", "LatestPrice"]
    missing = [k for k in req if not found.get(k)]
    if missing:
        raise ValueError(f"Iteration list missing {missing}. Found mapping: {found}. Columns: {list(df.columns)}")

    out = pd.DataFrame()
    out["TradeKey"]   = df[key_col].astype(str).str.strip()
    out["ORI_symbol"] = df[found["ORI_symbol"]].apply(normalize_symbol)
    out["Status"]     = df[found["Status"]].astype(str).str.strip().str.upper().apply(status_bucket)

    # latest price (used ONLY if trade is new / no state)
    out["LatestPrice"] = pd.to_numeric(df[found["LatestPrice"]], errors="coerce")

    if found.get("Direction"):
        out["Direction"] = df[found["Direction"]].apply(normalize_direction)
        out.loc[~out["Direction"].isin(["LONG","SHORT"]), "Direction"] = DEFAULT_DIRECTION
    else:
        out["Direction"] = DEFAULT_DIRECTION

    out["EntryDate_Assumed"] = yesterday_date()

    out = out[out["TradeKey"].notna() & (out["TradeKey"].str.len() > 0)].copy()
    out = out[out["ORI_symbol"].notna() & (out["ORI_symbol"].str.len() > 0)].copy()
    out = out[out["LatestPrice"].notna()].copy()
    out = out.sort_values(["TradeKey"]).drop_duplicates(subset=["TradeKey"], keep="last").reset_index(drop=True)

    log(f"Iteration trades loaded: {len(out)}")
    return out

# ============================================================
# Price files -> 15T OHLC + ATR (cached w/ mtime refresh)
# ============================================================

PRICE_COL_CANDS = {
    "date":  ["date", "Date", "datetime", "Datetime", "timestamp", "Timestamp"],
    "open":  ["open", "Open"],
    "high":  ["high", "High"],
    "low":   ["low", "Low"],
    "close": ["close", "Close"],
}

def resolve_price_file(symbol: str) -> str | None:
    sym = normalize_symbol(symbol)
    for ext in [".csv", ".CSV", ".parquet", ".PARQUET"]:
        fp = os.path.join(PRICE_DIR, f"{sym}{ext}")
        if os.path.exists(fp):
            return fp
    for fn in os.listdir(PRICE_DIR):
        if fn.upper().startswith(sym + "."):
            fp = os.path.join(PRICE_DIR, fn)
            if os.path.isfile(fp):
                return fp
    return None

def load_price_file(fp: str) -> pd.DataFrame:
    df = pd.read_parquet(fp) if fp.lower().endswith(".parquet") else pd.read_csv(fp)
    df.columns = [c.strip() for c in df.columns]
    cols = {k: find_column(df, v) for k, v in PRICE_COL_CANDS.items()}
    missing = [k for k, v in cols.items() if v is None]
    if missing:
        raise ValueError(f"Price file {os.path.basename(fp)} missing {missing}. Columns: {list(df.columns)}")

    out = pd.DataFrame()
    out["_dt"] = pd.to_datetime(df[cols["date"]], errors="coerce")
    for c in ["open","high","low","close"]:
        out[c] = pd.to_numeric(df[cols[c]], errors="coerce")
    out = out.dropna(subset=["_dt","open","high","low","close"]).sort_values("_dt").reset_index(drop=True)
    return out

def filter_market_hours(df: pd.DataFrame) -> pd.DataFrame:
    if df.empty:
        return df
    dfi = df.set_index("_dt", drop=False).sort_index()
    dfi = dfi.between_time(MARKET_START, MARKET_END, inclusive="both")
    return dfi.reset_index(drop=True)

def resample_ohlc(df: pd.DataFrame, rule: str) -> pd.DataFrame:
    if df.empty:
        return df
    dfi = df.set_index("_dt").sort_index()
    ohlc = dfi[["open","high","low","close"]].resample(rule).agg({
        "open": "first",
        "high": "max",
        "low": "min",
        "close": "last",
    }).dropna()
    return ohlc.reset_index()

def compute_atr(df: pd.DataFrame, period: int) -> pd.Series:
    high = df["high"]
    low = df["low"]
    close = df["close"]
    tr = pd.concat([
        high - low,
        (high - close.shift(1)).abs(),
        (low - close.shift(1)).abs()
    ], axis=1).max(axis=1)
    return tr.rolling(period, min_periods=period).mean()

_RS_CACHE = {}  # symbol -> {"rs": df, "mtime": float}

def prepare_rs_for_symbol(symbol: str) -> pd.DataFrame | None:
    sym = normalize_symbol(symbol)
    fp = resolve_price_file(sym)
    if fp is None:
        _RS_CACHE[sym] = {"rs": None, "mtime": None}
        return None

    mtime = os.path.getmtime(fp)
    cached = _RS_CACHE.get(sym)

    # refresh cache if file changed
    if cached and cached.get("mtime") == mtime and cached.get("rs") is not None:
        return cached["rs"]

    df = load_price_file(fp)
    df = filter_market_hours(df)
    if df.empty:
        _RS_CACHE[sym] = {"rs": None, "mtime": mtime}
        return None

    rs = resample_ohlc(df, RESAMPLE_RULE)
    if rs.empty:
        _RS_CACHE[sym] = {"rs": None, "mtime": mtime}
        return None

    rs["ATR"] = compute_atr(rs, ATR_PERIOD)
    _RS_CACHE[sym] = {"rs": rs, "mtime": mtime}
    return rs

# ============================================================
# Stop logic
# ============================================================

def adaptive_stop_distance_pct(bars: pd.DataFrame,
                               atan_threshold: float,
                               atr_fallback_mult: float,
                               min_stop_pct: float,
                               max_stop_pct: float,
                               sensitivity: float) -> float:
    if len(bars) < REG_WINDOW or bars["ATR"].dropna().empty:
        return np.nan

    recent = bars["close"].iloc[-REG_WINDOW:].astype(float)
    if len(recent) < REG_WINDOW:
        return np.nan

    x = np.arange(len(recent), dtype=float)
    a, b, _c = np.polyfit(x, recent.values, 2)

    slope_norm = (2 * a * x[-1] + b) / recent.iloc[-1]
    accel = (2 * a)
    atan_slope = float(np.arctan(slope_norm))

    atr = bars["ATR"].iloc[-1]
    px = bars["close"].iloc[-1]
    atr_pct = float(atr / px) if (pd.notna(atr) and atr > 0 and px > 0) else np.nan
    if pd.isna(atr_pct):
        return np.nan

    if abs(atan_slope) > atan_threshold:
        stop_pct = abs(slope_norm) * sensitivity * (1.0 + abs(accel))
        return float(np.clip(stop_pct, min_stop_pct, max_stop_pct))

    stop_pct = atr_pct * atr_fallback_mult
    return float(np.clip(stop_pct, min_stop_pct, max_stop_pct))

def compute_initial_stop_pct(rs: pd.DataFrame, direction: str) -> float:
    is_long = (normalize_direction(direction) == "LONG")
    atan_thr = CFG["ATAN_LONG"] if is_long else CFG["ATAN_SHORT"]
    max_stop = CFG["MAX_STOP_LONG"] if is_long else CFG["MAX_STOP_SHORT"]
    sens     = CFG["SENS_LONG"] if is_long else CFG["SENS_SHORT"]

    stop_pct = adaptive_stop_distance_pct(
        rs, atan_thr, ATR_FALLBACK_MULT,
        INITIAL_MIN_STOP_PCT, max_stop, sens
    )
    if pd.notna(stop_pct):
        return float(stop_pct)

    return float(np.clip(INITIAL_MIN_STOP_PCT, INITIAL_MIN_STOP_PCT, max_stop))

def init_stop_from_entry(entry_px: float, direction: str, stop_pct: float) -> float:
    is_long = (normalize_direction(direction) == "LONG")
    return entry_px * (1.0 - stop_pct) if is_long else entry_px * (1.0 + stop_pct)

def compute_monitor_start(entry_dt: pd.Timestamp) -> pd.Timestamp:
    if not APPLY_FROM_NEXT_DAY_ONLY:
        return entry_dt
    return entry_dt.normalize() + timedelta(days=1) + _todelta_hhmm(MARKET_START)

# ============================================================
# State history
# ============================================================

STATE_COLS = [
    "TradeKey","ORI_symbol","Direction",
    "EntryDate","EntryPrice",
    "Status","StopPrice","StopPct","StopSource",
    "BestRefPrice","LastBarDT"
]

def load_state_history(path: str) -> pd.DataFrame:
    if not os.path.exists(path):
        return pd.DataFrame(columns=STATE_COLS)
    df = pd.read_csv(path)
    df.columns = [c.strip() for c in df.columns]
    if "EntryDate" in df.columns:
        df["EntryDate"] = pd.to_datetime(df["EntryDate"], errors="coerce")
    if "LastBarDT" in df.columns:
        df["LastBarDT"] = pd.to_datetime(df["LastBarDT"], errors="coerce")
    return df

def save_state_history(df: pd.DataFrame, path: str):
    df.to_csv(path, index=False)

# ============================================================
# Actions model
# ============================================================

@dataclass
class Action:
    TradeKey: str
    ORI_symbol: str
    ActionType: str      # INIT_STOP / UPDATE_STOP / EXIT / NO_ACTION
    ActionTime: str
    StopPrice: float | None
    StopPct: float | None
    Reason: str

# ============================================================
# Per-trade step
# ============================================================

def process_trade_one_step(tr_state: pd.Series, rs: pd.DataFrame) -> tuple[pd.Series, list[Action]]:
    actions = []
    tkey = tr_state["TradeKey"]
    sym  = tr_state["ORI_symbol"]
    direction = normalize_direction(tr_state["Direction"])
    is_long = (direction == "LONG")

    entry_dt = pd.to_datetime(tr_state["EntryDate"])
    entry_px = float(tr_state["EntryPrice"])

    stop_price = float(tr_state["StopPrice"])
    stop_pct   = float(tr_state["StopPct"]) if pd.notna(tr_state["StopPct"]) else np.nan
    stop_src   = str(tr_state["StopSource"]) if pd.notna(tr_state["StopSource"]) else "Initial"
    best_ref   = float(tr_state["BestRefPrice"]) if pd.notna(tr_state["BestRefPrice"]) else entry_px
    last_dt    = pd.to_datetime(tr_state["LastBarDT"]) if pd.notna(tr_state["LastBarDT"]) else None

    rs = rs.sort_values("_dt").reset_index(drop=True)

    start_dt = compute_monitor_start(entry_dt)
    new_bars = rs[rs["_dt"] >= start_dt].copy() if last_dt is None else rs[rs["_dt"] > last_dt].copy()

    if new_bars.empty:
        actions.append(Action(tkey, sym, "NO_ACTION", now_ts(), stop_price, stop_pct, "No new bars"))
        return tr_state, actions

    exited = False
    updated = False
    last_processed = None

    for dt_val in new_bars["_dt"].tolist():
        idx = rs.index[rs["_dt"] == dt_val]
        if len(idx) == 0:
            continue
        i = int(idx[0])
        sub = rs.iloc[:i+1]
        row = rs.iloc[i]
        dt = row["_dt"]
        last_processed = dt

        close = float(row["close"])
        high  = float(row["high"])
        low   = float(row["low"])

        # update best_ref
        if is_long:
            best_ref = max(best_ref, close)
            hit_val = close if TRIGGER_ON_CLOSE else low
            if hit_val <= stop_price:
                tr_state["Status"] = "CLOSED"
                tr_state["LastBarDT"] = dt
                actions.append(Action(tkey, sym, "EXIT", str(dt), stop_price, stop_pct, f"Stop hit ({stop_src})"))
                exited = True
                break
        else:
            best_ref = min(best_ref, close)
            hit_val = close if TRIGGER_ON_CLOSE else high
            if hit_val >= stop_price:
                tr_state["Status"] = "CLOSED"
                tr_state["LastBarDT"] = dt
                actions.append(Action(tkey, sym, "EXIT", str(dt), stop_price, stop_pct, f"Stop hit ({stop_src})"))
                exited = True
                break

        # trailing update
        atan_thr = CFG["ATAN_LONG"] if is_long else CFG["ATAN_SHORT"]
        max_stop = CFG["MAX_STOP_LONG"] if is_long else CFG["MAX_STOP_SHORT"]
        sens     = CFG["SENS_LONG"] if is_long else CFG["SENS_SHORT"]

        new_stop_pct = adaptive_stop_distance_pct(
            sub, atan_thr, ATR_FALLBACK_MULT,
            TRAILING_MIN_STOP_PCT, max_stop, sens
        )
        if pd.isna(new_stop_pct):
            continue

        candidate_stop = (best_ref * (1.0 - new_stop_pct)) if is_long else (best_ref * (1.0 + new_stop_pct))

        if is_long and candidate_stop > stop_price:
            stop_price = float(candidate_stop)
            stop_pct = float(new_stop_pct)
            stop_src = "Trailing"
            updated = True
        elif (not is_long) and candidate_stop < stop_price:
            stop_price = float(candidate_stop)
            stop_pct = float(new_stop_pct)
            stop_src = "Trailing"
            updated = True

    tr_state["StopPrice"] = stop_price
    tr_state["StopPct"] = stop_pct
    tr_state["StopSource"] = stop_src
    tr_state["BestRefPrice"] = best_ref
    if last_processed is not None:
        tr_state["LastBarDT"] = last_processed

    if exited:
        return tr_state, actions

    if updated:
        actions.append(Action(tkey, sym, "UPDATE_STOP", now_ts(), stop_price, stop_pct, "Stop updated"))
    else:
        actions.append(Action(tkey, sym, "NO_ACTION", now_ts(), stop_price, stop_pct, "Stop unchanged"))

    return tr_state, actions

# ============================================================
# One cycle (runs every 15 min)
# ============================================================

def run_one_cycle():
    log("Cycle start")

    iter_df = load_iteration_df(ITERATION_FILE_PATH)
    st_hist = load_state_history(STATE_HISTORY_PATH)

    # refresh RS cache for all symbols (mtime-based)
    missing_files = []
    for sym in sorted(iter_df["ORI_symbol"].unique().tolist()):
        if resolve_price_file(sym) is None:
            missing_files.append(sym)
        prepare_rs_for_symbol(sym)

    if missing_files:
        log(f"WARNING missing price files for symbols (first 25): {missing_files[:25]}")

    state_rows = []
    actions_all = []

    for _, tr in iter_df.iterrows():
        tkey = tr["TradeKey"]
        sym  = tr["ORI_symbol"]
        direction = tr["Direction"]
        status = tr["Status"]

        rs = prepare_rs_for_symbol(sym)
        if rs is None or rs.empty:
            actions_all.append(Action(tkey, sym, "NO_ACTION", now_ts(), None, None, "Missing/empty price data"))
            continue

        prev = st_hist[st_hist["TradeKey"] == tkey].copy()

        if not prev.empty:
            # EXISTING STATE: DO NOT overwrite EntryPrice
            row = prev.iloc[-1].copy()
            row["ORI_symbol"] = sym
            row["Status"] = status
            row["Direction"] = direction if direction in ["LONG","SHORT"] else row.get("Direction", DEFAULT_DIRECTION)

            if pd.isna(row.get("EntryDate", pd.NaT)):
                row["EntryDate"] = yesterday_date()

        else:
            # NEW TRADE: set EntryPrice ONCE from latest price, EntryDate assumed yesterday
            entry_px = float(tr["LatestPrice"])
            row = pd.Series({
                "TradeKey": tkey,
                "ORI_symbol": sym,
                "Direction": direction if direction in ["LONG","SHORT"] else DEFAULT_DIRECTION,
                "EntryDate": yesterday_date(),
                "EntryPrice": entry_px,
                "Status": status,
                "StopPrice": np.nan,
                "StopPct": np.nan,
                "StopSource": "",
                "BestRefPrice": entry_px,
                "LastBarDT": pd.NaT
            })

        # INIT initial stop if missing (>=1%)
        if pd.isna(row.get("StopPrice", np.nan)):
            stop_pct = compute_initial_stop_pct(rs, row["Direction"])
            stop_price = init_stop_from_entry(float(row["EntryPrice"]), row["Direction"], stop_pct)

            row["StopPrice"] = float(stop_price)
            row["StopPct"] = float(stop_pct)
            row["StopSource"] = "Initial"
            row["BestRefPrice"] = float(row["EntryPrice"])
            row["LastBarDT"] = pd.NaT

            actions_all.append(Action(
                tkey, sym, "INIT_STOP", now_ts(),
                float(stop_price), float(stop_pct),
                "Initial stop (>=1%)"
            ))

        # process trailing / exit
        if status_bucket(row["Status"]) != "CLOSED":
            row2, acts = process_trade_one_step(row.copy(), rs)
            actions_all.extend(acts)
            state_rows.append(row2)
        else:
            state_rows.append(row)

    updated_state_df = pd.DataFrame(state_rows)

    # merge: replace OPEN/ACTIVE rows present today; keep historical CLOSED
    if not st_hist.empty and not updated_state_df.empty:
        open_active_mask = st_hist["Status"].astype(str).str.upper().isin(["OPEN","ACTIVE"])
        st_hist = st_hist[~(open_active_mask & st_hist["TradeKey"].isin(updated_state_df["TradeKey"]))].copy()

    final_state = pd.concat([st_hist, updated_state_df], ignore_index=True)
    save_state_history(final_state, STATE_HISTORY_PATH)

    actions_df = pd.DataFrame([a.__dict__ for a in actions_all])
    state_snapshot = final_state.sort_values(["Status","ORI_symbol","TradeKey"]).reset_index(drop=True)

    # Write ONE excel that keeps getting overwritten (latest view)
    with pd.ExcelWriter(LIVE_XLSX_PATH, engine="openpyxl", mode="w") as writer:
        actions_df.to_excel(writer, sheet_name="Actions", index=False)
        state_snapshot.to_excel(writer, sheet_name="StateSnapshot", index=False)

    log(f"Cycle done. Excel updated: {LIVE_XLSX_PATH}")

# ============================================================
# Main loop
# ============================================================

def main():
    log("Live Trailing Runner start")
    log(f"ITERATION_FILE_PATH={ITERATION_FILE_PATH}")
    log(f"PRICE_DIR={PRICE_DIR}")
    log(f"STATE_HISTORY_PATH={STATE_HISTORY_PATH}")
    log(f"LIVE_XLSX_PATH={LIVE_XLSX_PATH}")
    log(f"MODE={MODE}, POLL_SECONDS={POLL_SECONDS}")

    if MODE == "once":
        run_one_cycle()
        return

    while True:
        try:
            run_one_cycle()
        except Exception as e:
            log(f"ERROR: {repr(e)}")
        time.sleep(POLL_SECONDS)

if __name__ == "__main__":
    main()


[2026-01-24 00:10:43] Live Trailing Runner start
[2026-01-24 00:10:43] ITERATION_FILE_PATH=D:\work\Trade Analysis\Mapping ExERCISE\Iteration_list_2026_1_20_20.xlsx
[2026-01-24 00:10:43] PRICE_DIR=D:\work\Trade Analysis\Mapping ExERCISE\Latest_Data_L
[2026-01-24 00:10:43] STATE_HISTORY_PATH=D:\work\Trade Analysis\Mapping ExERCISE\state_history.csv
[2026-01-24 00:10:43] LIVE_XLSX_PATH=D:\work\Trade Analysis\Mapping ExERCISE\output\live_actions.xlsx
[2026-01-24 00:10:43] MODE=loop, POLL_SECONDS=900
[2026-01-24 00:10:43] Cycle start
[2026-01-24 00:10:43] Iteration trades loaded: 329


C:\Users\avina\AppData\Local\Temp\ipykernel_33432\4050304874.py:230: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ohlc = dfi[["open","high","low","close"]].resample(rule).agg({
C:\Users\avina\AppData\Local\Temp\ipykernel_33432\4050304874.py:230: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ohlc = dfi[["open","high","low","close"]].resample(rule).agg({
C:\Users\avina\AppData\Local\Temp\ipykernel_33432\4050304874.py:230: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ohlc = dfi[["open","high","low","close"]].resample(rule).agg({
C:\Users\avina\AppData\Local\Temp\ipykernel_33432\4050304874.py:230: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  ohlc = dfi[["open","high","low","close"]].resample(rule).agg({
C:\Users\avina\AppData\Local\Temp\ipykernel_33432\4050304874.py:

[2026-01-24 00:10:45] Cycle done. Excel updated: D:\work\Trade Analysis\Mapping ExERCISE\output\live_actions.xlsx
[2026-01-24 00:25:45] Cycle start
[2026-01-24 00:25:45] Iteration trades loaded: 329
[2026-01-24 00:25:46] Cycle done. Excel updated: D:\work\Trade Analysis\Mapping ExERCISE\output\live_actions.xlsx
[2026-01-24 00:40:46] Cycle start
[2026-01-24 00:40:47] Iteration trades loaded: 329
[2026-01-24 00:40:48] Cycle done. Excel updated: D:\work\Trade Analysis\Mapping ExERCISE\output\live_actions.xlsx
[2026-01-24 00:55:48] Cycle start
[2026-01-24 00:55:48] Iteration trades loaded: 329
[2026-01-24 00:55:50] Cycle done. Excel updated: D:\work\Trade Analysis\Mapping ExERCISE\output\live_actions.xlsx
[2026-01-24 01:10:50] Cycle start
[2026-01-24 01:10:50] Iteration trades loaded: 329
[2026-01-24 01:10:51] Cycle done. Excel updated: D:\work\Trade Analysis\Mapping ExERCISE\output\live_actions.xlsx
[2026-01-24 01:25:51] Cycle start
[2026-01-24 01:25:52] Iteration trades loaded: 329
[2026